In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Análise de dados dos Filmes e Programas de TV da Netflix

![](https://pngimg.com/uploads/netflix/netflix_PNG28.png)

> A Netflix é um serviço de transmissão online que oferece uma ampla variedade de séries, filmes e documentários premiados em milhares de aparelhos conectados à internet.

Fonte: https://www.netflix.com/br/


## **Integrantes desse projeto:**

**Julia Del Giudice** - https://www.linkedin.com/in/julia-del-giudice/

**Tatiana Martins**- https://www.linkedin.com/in/tatiims/

## **Objetivo**

Construímos essa análise de dados em conjunto com o intuito de aprender sobre a lingugem Python, bem como, conhecer as bibliotecas para modular, manipular e limpar os dados desse dataset. Dessa forma, trazendo informações e conhecimentos a partir desse conjunto de dados.

## **Obtenção dos Dados**

Os dados foram obtidos a partir do dataset "Netflix Movies and TV Shows".
Este conjunto de dados lista filmes e programas de TV disponíveis na Netflix até meados de 2021.

https://www.kaggle.com/datasets/shivamb/netflix-shows?datasetId=434238&select=netflix_titles.csv

In [ ]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [ ]:
#Lendo o arquivo:
df_netflix = pd.read_csv('../input/netflix-shows/netflix_titles.csv')

## **Análise dos Dados**
Nesta etapa nós vamos entender mais sobre os dados.

**Dicionário das variavéis**
- show_id: ID’s dos filmes e programas de TV 	
- type: Categoria por filmes ou programas de TV
- title: Nome do títulos
- director: Diretores 	
- cast: Elenco	
- country: País	
- date_added: Data que o filme foi adicionado na Netflix	
- release_year: Ano do lançamento dos títulos 	
- rating: Classificação por idade	
- duration: Duração	
- listed_in: Lista dos tipos de gêneros de filmes e programas de TV	
- description: Sinopse


> Na categoria programas de TV incluem: Séries, documentários, reality show, shows, etc.

In [ ]:
#Primeiras 5 linhas do Dataframe:
df_netflix.head()

In [ ]:
#Informações sobre o dataframe
df_netflix.info()

## **Limpeza e Transformação dos Dados**

In [ ]:
#Tratando valores nulos:
df_netflix['director'].fillna('Unknown', inplace = True)
df_netflix['cast'].fillna('Unknown', inplace = True)
df_netflix['country'].fillna('Unknown', inplace = True)
#Remove os valores nulos:
df_netflix.dropna(subset=['date_added','rating','duration'], inplace=True)

In [ ]:
#Verificando valores duplicados
if any(df_netflix.title.duplicated()) is True:
   print('Existe valores duplicados na coluna "title"')
else:
   print('Não existe valores duplicados na coluna "title"')

In [ ]:
df_netflix.info()

In [ ]:
df_netflix.head()

## **Análise Exploratória dos Dados**

**Entre Filmes e Programas de televisão, qual categoria tem o maior número de títulos?**

In [ ]:
#Total de filmes e programas de televisão
df_type = df_netflix.type.value_counts().to_frame()
print(df_type)

In [ ]:
fig1 = df_type.plot.pie(y ='type', autopct = '%0.01f%%', shadow=True, figsize=(10,8))
plt.title('Movie X TV Show', size=20)
plt.show(fig1)

**Entre Filmes e Programas de televisão, qual categoria tem o maior número de títulos por ano?**

In [ ]:
df_year = df_netflix.release_year.value_counts().sort_index(ascending = False)
df_movie_year = df_netflix[df_netflix.type == 'Movie'].release_year.value_counts().sort_index(ascending = False).to_frame()
df_tvshow_year = df_netflix[df_netflix.type == 'TV Show'].release_year.value_counts().sort_index(ascending = False).to_frame()
df_movie_year.columns = ['Movies']
df_tvshow_year.columns = ['Tv_Shows']
print(df_year.to_frame(),'\n----------\n',df_movie_year,'\n----------\n',df_tvshow_year)

In [ ]:
join = df_movie_year.join(df_tvshow_year,how='outer').sort_index(ascending = False)
join.fillna(0, inplace = True)
print(join)

In [ ]:
fig2 = join.head(31).plot(kind='bar', stacked=True, figsize=(10,8))
plt.xlabel('Ano')
plt.ylabel('Quantidade')
plt.title('Filmes X Programas de TV por ano', size=20)
plt.show(fig2)

**Entre Filmes e Programas de televisão, qual categoria tem o maior número de títulos por país?**

In [ ]:
df_country = df_netflix.country.value_counts()
df_movie_country = df_netflix[df_netflix.type == 'Movie'].country.value_counts().to_frame()
df_tvshow_country = df_netflix[df_netflix.type == 'TV Show'].country.value_counts().to_frame()
df_movie_country.columns = ['Movies']
df_tvshow_country.columns = ['Tv_Shows']
print(df_country.to_frame(),'\n----------\n',df_movie_country,'\n----------\n',df_tvshow_country)

**Quais são os Top 10 atores que mais participaram de Filmes?**

In [ ]:
top_actors = pd.DataFrame(df_netflix.cast.str.split(',').tolist(), index=df_netflix.type).stack()
top_actors = top_actors.reset_index([0, 'type'])
top_actors.columns = ['type', 'actor']

In [ ]:
top_actors.loc[(top_actors["type"]=='Movie') & (top_actors["actor"] != 'Unknown')]["actor"].value_counts().head(10).to_frame()

**Quais são os Top 10 atores que mais participaram de Programas de TV?**

In [ ]:
top_actors.loc[(top_actors["type"]=='TV Show') & (top_actors["actor"] != 'Unknown')]["actor"].value_counts().head(10).to_frame()

**Nuvem de palavras referente a categoria dos títulos**

In [ ]:
df_list = pd.DataFrame(df_netflix.listed_in.str.split(',').tolist(), index=df_netflix.type).stack()
df_list = df_list.reset_index([0, 'type'])
df_list.columns = ['type', 'list']

In [ ]:
df_list.loc[(df_list["list"] !="unknown")]["list"].value_counts().head(20) 


In [ ]:
df_list_in = df_list.loc[(df_list["list"] !="unknown")]["list"]
print(df_list_in)

In [ ]:
wordcloud = WordCloud(max_font_size=350, collocations=False,width=1920, height=1080, background_color="black").generate(' '.join(df_list_in))
plt.figure(figsize=(12,8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

## **Conclusão**

Ao fazer a análise desse conjunto de dados compreendemos que na categoria programas de TV incluem: Séries, documentários, reality show, shows, etc. Até meados de 2021 o catálogo da Netflix era composto por mais filmes do que programas de TV. 

No ano de 2015 os programas de TV tiveram um aumento significativo e os Estados Unidos é o país que lidera os lançamentos tanto por filmes e programas de TV, mas apesar dessa liderança do país os atores que mais aparecem são do continente asiático.